visdom example


In [1]:
import torch
import torchvision
import visdom

In [2]:
vis = visdom.Visdom()

Setting up a new session...


In [3]:
vis.text("Hello world1!", env="main") # 텍스트 띄우기

'window_3a699501750ec6'

In [4]:
a = torch.randn(3, 200, 200)
vis.image(a) # 이미지 띄우기

'window_3a69950182552a'

In [5]:
vis.images(torch.randn(3, 3, 28, 28))

'window_3a699501978ad2'

example(MSIST, CIFAR10)

In [6]:
mnist = torchvision.datasets.MNIST(root="./MNIST_data", train=True, transform=torchvision.transforms.ToTensor(), download=True)
cifar10 = torchvision.datasets.CIFAR10(root="./cifar10", train=True, transform=torchvision.transforms.ToTensor(), download=True)


Files already downloaded and verified


In [7]:
data = cifar10.__getitem__(0)
print(data[0].shape)
vis.images(data[0], env="main")

torch.Size([3, 32, 32])


'window_3a6995026f4d40'

In [8]:
data = mnist.__getitem__(0)
print(data[0].shape)
vis.images(data[0], env="main")

torch.Size([1, 28, 28])


'window_3a6995027fc8e8'

In [9]:
data_loader = torch.utils.data.DataLoader(dataset=mnist, batch_size=32, shuffle=False)


In [10]:
for num, value in enumerate(data_loader):
    value = value[0]
    print(value.shape)
    vis.images(value)
    break

torch.Size([32, 1, 28, 28])


In [11]:
vis.close(env="main")

''

line plot

In [12]:
Y_data = torch.randn(5)
X_data = torch.Tensor([1, 2, 3, 4, 5])
plt = vis.line(Y=Y_data, X=X_data)

line update

In [13]:
Y_append = torch.randn(1)
X_append = torch.Tensor([6])

vis.line(Y=Y_append, X=X_append, win=plt, update="append")

'window_3a6995029e9c16'

multiple line on single windows

In [14]:
num = torch.Tensor(list(range(0, 10)))
num = num.view(-1, 1)
num = torch.cat((num, num), dim=1)

plt = vis.line(Y=torch.randn(10, 2), X=num)

line info

In [15]:
plt = vis.line(Y=Y_data, X=X_data, opts=dict(title="TEST", showlegend=True, legend=["1번"]))
# title : 그래프 이름, showlenend : 범례, legend : 축 이름

In [16]:
plt = vis.line(Y=torch.randn(10, 2), X=num, opts=dict(title="TEST", showlegend=True, legend=["1번", "2번"]))
# 축이 2개 이상일때는 legend를 리스트로 일렬로 추가해주면 됨

In [17]:
def loss_tracker(loss_plot, loss_value, num):
    ''' num, loss_value are Tensor '''
    vis.line(X=num, Y=loss_value, win=loss_plot, update="append")

In [18]:
plt = vis.line(Y=torch.Tensor(1).zero_())

for i in range(500):
    loss = torch.randn(1) + i
    loss_tracker(plt, loss, torch.Tensor([i]))

In [19]:
vis.close(env="main")

''

MNIST-CNN with Visdom

In [20]:
def loss_tracker(loss_plot, loss_value, num):
    ''' num, loss_value are Tensor '''
    vis.line(X=num, Y=loss_value, win=loss_plot, update="append")

In [21]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [22]:
learning_rate = 0.001
training_epochs = 15
batch_size = 32

In [23]:
mnist_train = torchvision.datasets.MNIST(root='MNIST_data', train=True, transform=torchvision.transforms.ToTensor(), download=True)
mnist_test = torchvision.datasets.MNIST(root='MNIST_data', train=False, transform=torchvision.transforms.ToTensor(), download=True)


In [24]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)

In [25]:
class CNN(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2)
        )
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2)
        )
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2)
        )
        self.fc1 = torch.nn.Linear(3*3*128, 625, bias=True)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)

        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        
        return out


In [26]:
model = CNN().to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
loss_plt = vis.line(Y=torch.Tensor(1).zero_(),opts=dict(title='loss_tracker', legend=['loss'], showlegend=True))

In [27]:
total_batch = len(data_loader)

for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)

        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost/total_batch
    
    print('[Epoch:{:2d}] cost = {}'.format(epoch+1, avg_cost))
    loss_tracker(loss_plt, torch.Tensor([avg_cost]), torch.Tensor([epoch]))
print("Learning Finished")

[Epoch: 1] cost = 0.12909138202667236
[Epoch: 2] cost = 0.041219018399715424
[Epoch: 3] cost = 0.029452497139573097
[Epoch: 4] cost = 0.02253258414566517
[Epoch: 5] cost = 0.017607921734452248
[Epoch: 6] cost = 0.014917848631739616
[Epoch: 7] cost = 0.013245399110019207
[Epoch: 8] cost = 0.01082478929311037
[Epoch: 9] cost = 0.008571264334022999
[Epoch:10] cost = 0.008940220810472965
[Epoch:11] cost = 0.008443604223430157
[Epoch:12] cost = 0.006789905484765768
[Epoch:13] cost = 0.008836997672915459
[Epoch:14] cost = 0.006068942602723837
[Epoch:15] cost = 0.006328559014946222
Learning Finished


In [28]:
with torch.no_grad():
    X_test = mnist_test.data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.targets.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean() 
    print('Accuracy:', accuracy.item())

Accuracy: 0.973099946975708


[WinError 10054] 현재 연결은 원격 호스트에 의해 강제로 끊겼습니다
on_close() takes 1 positional argument but 3 were given
[WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다
on_close() takes 1 positional argument but 3 were given
[WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다
on_close() takes 1 positional argument but 3 were given
[WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다
on_close() takes 1 positional argument but 3 were given
[WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다
on_close() takes 1 positional argument but 3 were given
[WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다
on_close() takes 1 positional argument but 3 were given
[WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다
on_close() takes 1 positional argument but 3 were given
[WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다
on_close() takes 1 positional argument but 3 were given
[WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다
on_close() takes 1 positional argument but 3 were given
[WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다
on_close() takes 1 